## Build a CNN for Image Recognition.
### Name: Joe Arul Susai Prakash (U34351756)
### For class ISM6930 (Tech Foundation of AI) at the University of South Florida

### Summary of Results :


- VGG16 architecture (__15 layers, 75 training epochs__) achieved __90.05%__ accuracy on test data with loss 0.317.
- ResNet architecture (__48 layers, 35 training epochs__) achieved __82%__ accuracy on training before I ran out of compute units and lost access to the runtime and the model.

## Data preparation

In [ ]:
import numpy as np
import tensorflow as tf

### Load data

In [ ]:
from keras.datasets import cifar10


(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('shape of x_train: ' + str(x_train.shape))
print('shape of y_train: ' + str(y_train.shape))
print('shape of x_test: ' + str(x_test.shape))
print('shape of y_test: ' + str(y_test.shape))
print('number of classes: ' + str(np.max(y_train) - np.min(y_train) + 1))

170498071/170498071 [==============================] - 6s 0us/step
shape of x_train: (50000, 32, 32, 3)
shape of y_train: (50000, 1)
shape of x_test: (10000, 32, 32, 3)
shape of y_test: (10000, 1)
number of classes: 10


### One-hot encode the labels

In [ ]:
def one_hot_encode(labels):
  num_labels = len(np.unique(labels))
  one_hot_labels = np.zeros((len(labels), num_labels))
  for i, label in enumerate(labels):
    one_hot_labels[i, label] = 1
  return one_hot_labels

In [ ]:
y_train_vec = one_hot_encode(y_train)
y_test_vec = one_hot_encode(y_test)

print('Shape of y_train_vec: ' + str(y_train_vec.shape))
print('Shape of y_test_vec: ' + str(y_test_vec.shape))

print(y_train[0])
print(y_train_vec[0])

Shape of y_train_vec: (50000, 10)
Shape of y_test_vec: (10000, 10)
[6]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


### Randomly partition the training set to training and validation sets

In [ ]:
rand_indices = np.random.permutation(50000)
train_indices = rand_indices[0:40000]
valid_indices = rand_indices[40000:50000]

x_val = x_train[valid_indices, :]
y_val = y_train_vec[valid_indices, :]

x_tr = x_train[train_indices, :]
y_tr = y_train_vec[train_indices, :]

print('Shape of x_tr: ' + str(x_tr.shape))
print('Shape of y_tr: ' + str(y_tr.shape))
print('Shape of x_val: ' + str(x_val.shape))
print('Shape of y_val: ' + str(y_val.shape))

Shape of x_tr: (40000, 32, 32, 3)
Shape of y_tr: (40000, 10)
Shape of x_val: (10000, 32, 32, 3)
Shape of y_val: (10000, 10)


## Build a CNN and tune its hyper-parameters

- Build a convolutional neural network model
- Use the validation data to tune the hyper-parameters
- Try to achieve a validation accuracy as high as possible

#### Model 1: Baseline model / LeNet Architecture (__10 epochs__)

- 2 Convolution and Pooling layers
- 1 Fully connected layer
- RMSProp optimizer with learning rate = 0.00001
- Batch size of train and val = 32

__Validation accuracy at last epoch : 48.7%__

__Validation Loss at last epoch : 1.61__

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential



model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 8, 8, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense (Dense)               (None, 128)               5

In [ ]:
from keras import optimizers

learning_rate = 1E-5 # to be tuned!

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_tr, y_tr, batch_size=32, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
1250/1250 [==============================] - 14s 4ms/step - loss: 6.3495 - accuracy: 0.2112 - val_loss: 3.9476 - val_accuracy: 0.2736
Epoch 2/10
1250/1250 [==============================] - 4s 3ms/step - loss: 3.3711 - accuracy: 0.3033 - val_loss: 2.9532 - val_accuracy: 0.3271
Epoch 3/10
1250/1250 [==============================] - 4s 3ms/step - loss: 2.6523 - accuracy: 0.3461 - val_loss: 2.4919 - val_accuracy: 0.3585
Epoch 4/10
1250/1250 [==============================] - 4s 3ms/step - loss: 2.2666 - accuracy: 0.3771 - val_loss: 2.1994 - val_accuracy: 0.3832
Epoch 5/10
1250/1250 [==============================] - 4s 3ms/step - loss: 2.0182 - accuracy: 0.4046 - val_loss: 1.9976 - val_accuracy: 0.4077
Epoch 6/10
1250/1250 [==============================] - 4s 3ms/step - loss: 1.8441 - accuracy: 0.4317 - val_loss: 1.8837 - val_accuracy: 0.4293
Epoch 7/10
1250/1250 [==============================] - 4s 3ms/step - loss: 1.7161 - accuracy: 0.4559 - val_loss: 1.8006 - val_accuracy

#### Model 2: Baseline model with Data Augmentation (__10 epochs__)

- 2 Convolution and Pooling layers
- 1 Fully connected layer
- __Data Augmentation on training data__
- RMSProp optimizer with learning rate = 0.0001
- Batch size of train and val = 32

__Validation accuracy at last epoch : 54%__

__Validation Loss at last epoch : 1.29__

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential

model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dense(10, activation='softmax'))


model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 8, 8, 64)          0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 4096)              0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [ ]:
from keras import optimizers

learning_rate = 1e-4

model2.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
train_datagen.fit(x_tr)
val_datagen.fit(x_val)

In [ ]:
history = model2.fit(train_datagen.flow(x_tr, y_tr, batch_size=32), validation_data=val_datagen.flow(x_val, y_val, batch_size=32), epochs=10)

Epoch 1/10
1250/1250 [==============================] - 30s 23ms/step - loss: 1.9580 - accuracy: 0.2964 - val_loss: 1.7268 - val_accuracy: 0.3893
Epoch 2/10
1250/1250 [==============================] - 28s 23ms/step - loss: 1.7571 - accuracy: 0.3695 - val_loss: 1.5946 - val_accuracy: 0.4368
Epoch 3/10
1250/1250 [==============================] - 28s 23ms/step - loss: 1.6834 - accuracy: 0.3959 - val_loss: 1.6905 - val_accuracy: 0.3981
Epoch 4/10
1250/1250 [==============================] - 28s 23ms/step - loss: 1.6242 - accuracy: 0.4196 - val_loss: 1.4885 - val_accuracy: 0.4751
Epoch 5/10
1250/1250 [==============================] - 28s 23ms/step - loss: 1.5865 - accuracy: 0.4359 - val_loss: 1.3894 - val_accuracy: 0.5129
Epoch 6/10
1250/1250 [==============================] - 29s 23ms/step - loss: 1.5511 - accuracy: 0.4453 - val_loss: 1.4701 - val_accuracy: 0.4863
Epoch 7/10
1250/1250 [==============================] - 29s 23ms/step - loss: 1.5164 - accuracy: 0.4586 - val_loss: 1.3087 -

#### Model 3: Baseline Model with Data Augmentation and Normalization layers (__10 epochs__)

- 2 Convolution and Pooling layers
- 1 Fully connected layer
- Data Augmentation on training data
- __Batch Normalization between hidden and activation layers__
- RMSProp optimizer with learning rate = 0.00001
- Batch size of train and val = 32

__Validation accuracy at last epoch : 50.5%__

__Validation Loss at last epoch : 1.417__

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras import layers

model_3 = Sequential()
model_3.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3)))
model_3.add(layers.BatchNormalization())
model_3.add(layers.Activation('relu'))
model_3.add(MaxPooling2D((2, 2)))
model_3.add(Conv2D(64, (3, 3),padding='same'))
model_3.add(layers.BatchNormalization())
model_3.add(layers.Activation('relu'))
model_3.add(MaxPooling2D((2, 2)))
model_3.add(Flatten())
model_3.add(Dense(128))
model_3.add(layers.BatchNormalization())
model_3.add(layers.Activation('relu'))
model_3.add(Dense(10, activation='softmax'))


model_3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 16, 16, 64)       

In [ ]:
from keras import optimizers

#learning_rate = 1e-4

learning_rate = 1e-5

model_3.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
train_datagen.fit(x_tr)
val_datagen.fit(x_val)

In [ ]:
history = model_3.fit(train_datagen.flow(x_tr, y_tr, batch_size=32), validation_data=val_datagen.flow(x_val, y_val, batch_size=8), epochs=10)

Epoch 1/10
1250/1250 [==============================] - 31s 23ms/step - loss: 2.0981 - accuracy: 0.2605 - val_loss: 1.8579 - val_accuracy: 0.3601
Epoch 2/10
1250/1250 [==============================] - 29s 23ms/step - loss: 1.8451 - accuracy: 0.3406 - val_loss: 1.7377 - val_accuracy: 0.3947
Epoch 3/10
1250/1250 [==============================] - 29s 23ms/step - loss: 1.7610 - accuracy: 0.3724 - val_loss: 1.6468 - val_accuracy: 0.4261
Epoch 4/10
1250/1250 [==============================] - 29s 23ms/step - loss: 1.7090 - accuracy: 0.3864 - val_loss: 1.5915 - val_accuracy: 0.4425
Epoch 5/10
1250/1250 [==============================] - 29s 23ms/step - loss: 1.6628 - accuracy: 0.4056 - val_loss: 1.5468 - val_accuracy: 0.4602
Epoch 6/10
1250/1250 [==============================] - 29s 23ms/step - loss: 1.6341 - accuracy: 0.4173 - val_loss: 1.5085 - val_accuracy: 0.4742
Epoch 7/10
1250/1250 [==============================] - 29s 23ms/step - loss: 1.6025 - accuracy: 0.4286 - val_loss: 1.4771 -

#### Model 4: Baseline Model with Data Augmentation, Normalization layers and Dropout layer (__25 epochs__)
- 2 Convolution and Pooling layers
- 1 Fully connected layer
- Data Augmentation on training data
- Batch Normalization between hidden and activation layers
- __Dropout 50% before dense layer (epochs - 25) to avoid overfitting__
- RMSProp optimizer with learning rate = 0.0001
- Batch size of train and val = 32

__Validation accuracy at last epoch : 56.3%__

__Validation Loss at last epoch : 1.254__

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras import layers

model_4 = Sequential()
model_4.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3)))
model_4.add(layers.BatchNormalization())
model_4.add(layers.Activation('relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Conv2D(64, (3, 3),padding='same'))
model_4.add(layers.BatchNormalization())
model_4.add(layers.Activation('relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Flatten())
model_4.add(layers.Dropout(0.5))
model_4.add(Dense(128))
model_4.add(layers.BatchNormalization())
model_4.add(layers.Activation('relu'))
model_4.add(Dense(10, activation='softmax'))


model_4.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_3 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 activation_3 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 batch_normalization_4 (Bat  (None, 16, 16, 64)       

In [ ]:
from keras import optimizers

#learning_rate = 1e-4

learning_rate = 1e-4

model_4.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
train_datagen.fit(x_tr)
val_datagen.fit(x_val)

In [ ]:
history = model_4.fit(train_datagen.flow(x_tr, y_tr), validation_data=val_datagen.flow(x_val, y_val),batch_size=32,epochs=25)

Epoch 1/25
1250/1250 [==============================] - 30s 22ms/step - loss: 1.9000 - accuracy: 0.3144 - val_loss: 1.6301 - val_accuracy: 0.4234
Epoch 2/25
1250/1250 [==============================] - 27s 22ms/step - loss: 1.7006 - accuracy: 0.3871 - val_loss: 1.5075 - val_accuracy: 0.4613
Epoch 3/25
1250/1250 [==============================] - 28s 22ms/step - loss: 1.6196 - accuracy: 0.4155 - val_loss: 1.4021 - val_accuracy: 0.4970
Epoch 4/25
1250/1250 [==============================] - 28s 22ms/step - loss: 1.5724 - accuracy: 0.4339 - val_loss: 1.3574 - val_accuracy: 0.5165
Epoch 5/25
1250/1250 [==============================] - 28s 23ms/step - loss: 1.5310 - accuracy: 0.4478 - val_loss: 1.3426 - val_accuracy: 0.5244
Epoch 6/25
1250/1250 [==============================] - 28s 22ms/step - loss: 1.5037 - accuracy: 0.4616 - val_loss: 1.2674 - val_accuracy: 0.5458
Epoch 7/25
1250/1250 [==============================] - 28s 22ms/step - loss: 1.4733 - accuracy: 0.4734 - val_loss: 1.3388 -

#### Model 5: __AlexNet Architecture (from scratch / 25 epochs)__
Reference: https://www.kaggle.com/code/vortexkol/alexnet-cnn-architecture-on-tensorflow-beginner

- 5 Convolution layers (2 with pooling)
- 2 Fully connected layers
- Data Augmentation on training data
- Batch Normalization between hidden and activation layers
- Dropout 50% before both dense layers (epochs - 25) to avoid overfitting
- Adam optimizer with learning rate = 0.001
- Batch size of train =32 and val = 8

__Validation accuracy at last epoch : 72.1%__

__Validation Loss at last epoch : 0.815__

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras import layers

model_5 = Sequential()
# Convolution 1 with pooling
model_5.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3)))
model_5.add(layers.BatchNormalization())
model_5.add(layers.Activation('relu'))
model_5.add(MaxPooling2D((2, 2)))

# Convolution 2 with pooling
model_5.add(Conv2D(64, (3, 3),padding='same'))
model_5.add(layers.BatchNormalization())
model_5.add(layers.Activation('relu'))
model_5.add(MaxPooling2D((3, 3)))

# Convolution 3
model_5.add(Conv2D(128, (3, 3),padding='same'))
model_5.add(layers.BatchNormalization())
model_5.add(layers.Activation('relu'))

#Convolution 4
model_5.add(Conv2D(256, (3, 3),padding='same'))
model_5.add(layers.BatchNormalization())
model_5.add(layers.Activation('relu'))

# Convolution 5
model_5.add(Conv2D(256, (3, 3),padding='same'))
model_5.add(layers.BatchNormalization())
model_5.add(layers.Activation('relu'))
model_5.add(MaxPooling2D((2, 2)))

model_5.add(Flatten())

# Fully connected layer 1
model_5.add(layers.Dropout(0.5))
model_5.add(Dense(1024))
model_5.add(layers.BatchNormalization())
model_5.add(layers.Activation('relu'))

# Fully connected layer 2
model_5.add(layers.Dropout(0.5))
model_5.add(Dense(128))
model_5.add(layers.BatchNormalization())
model_5.add(layers.Activation('relu'))

model_5.add(Dense(10, activation='softmax'))


model_5.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_6 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 activation_6 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 16, 16, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 batch_normalization_7 (Bat  (None, 16, 16, 64)       

In [ ]:
from keras import optimizers

#learning_rate = 1e-4

learning_rate = 1e-3

model_5.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
train_datagen.fit(x_tr)
val_datagen.fit(x_val)

In [ ]:
history = model_5.fit(train_datagen.flow(x_tr, y_tr, batch_size=32), validation_data=val_datagen.flow(x_val, y_val, batch_size=8),epochs=25)

Epoch 1/25
1250/1250 [==============================] - 37s 25ms/step - loss: 1.7353 - accuracy: 0.3674 - val_loss: 1.5262 - val_accuracy: 0.4362
Epoch 2/25
1250/1250 [==============================] - 31s 24ms/step - loss: 1.4485 - accuracy: 0.4778 - val_loss: 1.6675 - val_accuracy: 0.4682
Epoch 3/25
1250/1250 [==============================] - 30s 24ms/step - loss: 1.3214 - accuracy: 0.5276 - val_loss: 1.2632 - val_accuracy: 0.5614
Epoch 4/25
1250/1250 [==============================] - 31s 25ms/step - loss: 1.2332 - accuracy: 0.5640 - val_loss: 1.2158 - val_accuracy: 0.5649
Epoch 5/25
1250/1250 [==============================] - 31s 25ms/step - loss: 1.1726 - accuracy: 0.5869 - val_loss: 1.4052 - val_accuracy: 0.5441
Epoch 6/25
1250/1250 [==============================] - 30s 24ms/step - loss: 1.1216 - accuracy: 0.6079 - val_loss: 1.3327 - val_accuracy: 0.5568
Epoch 7/25
1250/1250 [==============================] - 31s 25ms/step - loss: 1.0712 - accuracy: 0.6237 - val_loss: 1.1926 -

In [ ]:
model_5.save('A2_Alexnet.keras')

In [ ]:
model_to_use = tf.keras.models.load_model('A2_Alexnet.keras')

#### Model 6: __VGG16 Architecture (from scratch / 25 epochs)__
Reference: https://builtin.com/machine-learning/vgg16

- 13 Convolution layers (5 with pooling)
- 2 Fully connected layers
- Data Augmentation on training data
- Batch Normalization between hidden and activation layers
- Dropout 50% before both dense layers (epochs - 25) to avoid overfitting
- Adam optimizer with learning rate = 0.001
- Batch size of train =32 and val = 8

__Validation accuracy at last epoch : 77%__

__Validation Loss at last epoch : 0.707__

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras import layers

model_6 = Sequential()
# Convolution 1 with pooling
model_6.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3)))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))

# Convolution 2 with pooling
model_6.add(Conv2D(64, (3, 3),padding='same'))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))
model_6.add(MaxPooling2D((2, 2))) # pooling after conv 1,2

# Convolution 3
model_6.add(Conv2D(128, (3, 3),padding='same'))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))

#Convolution 4
model_6.add(Conv2D(128, (3, 3),padding='same'))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))
model_6.add(MaxPooling2D((2, 2))) # pooling after conv 3,4

# Convolution 5
model_6.add(Conv2D(256, (3, 3),padding='same'))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))

# Convolution 6
model_6.add(Conv2D(256, (3, 3),padding='same'))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))

# Convolution 7
model_6.add(Conv2D(256, (3, 3),padding='same'))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))
model_6.add(MaxPooling2D((2, 2))) # pooling after conv 5,6,7

# Convolution 8
model_6.add(Conv2D(512, (3, 3),padding='same'))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))

# Convolution 9
model_6.add(Conv2D(512, (3, 3),padding='same'))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))

# Convolution 10
model_6.add(Conv2D(512, (3, 3),padding='same'))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))
model_6.add(MaxPooling2D((2, 2))) # pooling after conv 8,9,10

# Convolution 11
model_6.add(Conv2D(512, (3, 3),padding='same'))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))

# Convolution 12
model_6.add(Conv2D(512, (3, 3),padding='same'))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))

# Convolution 13
model_6.add(Conv2D(512, (3, 3),padding='same'))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))
model_6.add(MaxPooling2D((2, 2))) # pooling after conv 11,12,13

model_6.add(Flatten())

# Fully connected layer 1
model_6.add(layers.Dropout(0.5))
model_6.add(Dense(1024))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))

# Fully connected layer 2
model_6.add(layers.Dropout(0.5))
model_6.add(Dense(1024))
model_6.add(layers.BatchNormalization())
model_6.add(layers.Activation('relu'))

model_6.add(Dense(10, activation='softmax'))


model_6.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 64)        256       
 chNormalization)                                                
                                                                 
 activation_1 (Activation)   (None, 32, 32, 64)        0

In [ ]:
from keras import optimizers

#learning_rate = 1e-4

learning_rate = 1e-3

model_6.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
train_datagen.fit(x_tr)
val_datagen.fit(x_val)

In [ ]:
history = model_6.fit(train_datagen.flow(x_tr, y_tr, batch_size=32), validation_data=val_datagen.flow(x_val, y_val, batch_size=8),epochs=25)

Epoch 1/25
1250/1250 [==============================] - 49s 23ms/step - loss: 2.1811 - accuracy: 0.1837 - val_loss: 3.0783 - val_accuracy: 0.1199
Epoch 2/25
1250/1250 [==============================] - 29s 23ms/step - loss: 2.0506 - accuracy: 0.2076 - val_loss: 2.0024 - val_accuracy: 0.2111
Epoch 3/25
1250/1250 [==============================] - 28s 23ms/step - loss: 1.9319 - accuracy: 0.2603 - val_loss: 2.1671 - val_accuracy: 0.2360
Epoch 4/25
1250/1250 [==============================] - 29s 23ms/step - loss: 1.8293 - accuracy: 0.3094 - val_loss: 2.5702 - val_accuracy: 0.2464
Epoch 5/25
1250/1250 [==============================] - 28s 23ms/step - loss: 1.7264 - accuracy: 0.3566 - val_loss: 1.6632 - val_accuracy: 0.4279
Epoch 6/25
1250/1250 [==============================] - 29s 23ms/step - loss: 1.6444 - accuracy: 0.3905 - val_loss: 1.4879 - val_accuracy: 0.4311
Epoch 7/25
1250/1250 [==============================] - 28s 23ms/step - loss: 1.5087 - accuracy: 0.4509 - val_loss: 1.4183 -

In [ ]:
model_6.save('A2_VGG16_U34351756.keras')

#### __Model 7: ResNet (from scratch / 10 epochs)__
Reference : https://www.kaggle.com/code/mishki/resnet-keras-code-from-scratch-train-on-gpu
- __48 layers__ in total (including 4 convolutions for identity)
  - 3 layers in each identity block
  - 4 layers in each convolution block
  - 14 blocks stacked one by one
  - No pooling after each convolution
  - 1 FC layer with dropout
- Data Augmentation on training data
- Batch Normalization between hidden and activation layers
- Dropout 50% before the dense layer to avoid overfitting
- Adam optimizer with learning rate = 0.001
- Batch size of train =32 and val = 8

__Validation accuracy at last epoch : 63.4%__

__Validation Loss at last epoch : 1.065__

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_datagen.fit(x_tr)
val_datagen.fit(x_val)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, concatenate, GlobalAveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model, Sequential

In [ ]:
def identity_block(input_tensor, filter1, filter2, filter3):
    # layer 1 of block
    x = Conv2D(filter1, (1, 1), padding="same")(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # layer 2 of block
    x = Conv2D(filter2, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # layer 3 of block
    x = Conv2D(filter3, (1, 1), padding="same")(x)
    x = BatchNormalization()(x)

    # Add the residual connection
    x = concatenate([x, input_tensor])
    x = ReLU()(x)
    return x

In [ ]:
def convolution_block(input_tensor, filter1, filter2, filter3):
    # layer 1 of block
    x = Conv2D(filter1, (1, 1), padding="same")(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # layer 2 of block
    x = Conv2D(filter2, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # layer 3 of block
    x = Conv2D(filter3, (1, 1), padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # convolution for residual layer
    residual = Conv2D(filter3, (1, 1), padding="same")(input_tensor)
    residual = BatchNormalization()(residual)
    x = concatenate([x, residual])
    x = ReLU()(x)
    return x


In [ ]:

# Model initialization
model_7 = Sequential()

# Convolution 1 with pooling
model_7.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3)))
model_7.add(BatchNormalization())
model_7.add(ReLU())

# Level 1 (3 blocks)
x = convolution_block(model_7.layers[-1].output, 32, 32, 64)
x = identity_block(x, 64, 64, 128)
x = identity_block(x, 64, 64, 128)

# Level 2 (3 blocks)
x = convolution_block(x, 128, 128, 256)
x = identity_block(x, 128, 128, 256)
x = identity_block(x, 256, 256, 512)

# Level 3 (5 blocks)
x = convolution_block(x, 256, 256, 512)
x = identity_block(x, 256, 256, 512)
x = identity_block(x, 512, 512, 1024)
x = identity_block(x, 512, 512, 1024)
x = identity_block(x, 512, 512, 1024)

# Level 4 (3 blocks)
x = convolution_block(x, 512, 512, 1024)
x = identity_block(x, 512, 512, 1024)
x = identity_block(x, 512, 512, 1024)

# Final pooling
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)

# Fully connected layer 1
x = tf.keras.layers.Dropout(0.5)(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = ReLU()(x)

# Output Layer
output = Dense(10, activation='softmax')(x)

# Define the model
model_7 = Model(inputs=model_7.inputs, outputs=output)


In [ ]:
from keras import optimizers

#learning_rate = 1e-4

learning_rate = 1e-3

model_7.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
history = model_7.fit(train_datagen.flow(x_tr, y_tr, batch_size=32), validation_data=val_datagen.flow(x_val, y_val, batch_size=8),epochs=10)

Epoch 1/10
1250/1250 [==============================] - 272s 178ms/step - loss: 2.0002 - accuracy: 0.2768 - val_loss: 2.7585 - val_accuracy: 0.3240
Epoch 2/10
1250/1250 [==============================] - 221s 177ms/step - loss: 1.7562 - accuracy: 0.3687 - val_loss: 6.5839 - val_accuracy: 0.2529
Epoch 3/10
1250/1250 [==============================] - 221s 177ms/step - loss: 1.6550 - accuracy: 0.4073 - val_loss: 8.9791 - val_accuracy: 0.2239
Epoch 4/10
1250/1250 [==============================] - 221s 177ms/step - loss: 1.4910 - accuracy: 0.4590 - val_loss: 1.6410 - val_accuracy: 0.4454
Epoch 5/10
1250/1250 [==============================] - 221s 177ms/step - loss: 1.3790 - accuracy: 0.5026 - val_loss: 2.1400 - val_accuracy: 0.4088
Epoch 6/10
1250/1250 [==============================] - 222s 177ms/step - loss: 1.3052 - accuracy: 0.5297 - val_loss: 1.5484 - val_accuracy: 0.4819
Epoch 7/10
1250/1250 [==============================] - 222s 177ms/step - loss: 1.2409 - accuracy: 0.5556 - val_

In [ ]:
model_7.save('A2_ResNet_U34351756.keras')

## Train (again) and evaluate the model

### Train the model on the entire training set

#### __Based on previous iterations of training, VGG16 and ResNet architectures seem to perform better than other versions. I will be training both these models with the entire training set (changing 1 hyperparameter - epochs) and evaluate them on the test set__

#### VGG16 (model 6) Re-training with same architecture (epochs = 50)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# entire training set
train_datagen.fit(x_train)


In [ ]:
from keras import optimizers

#learning_rate = 1e-4

learning_rate = 1e-3

model_6.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
history = model_6.fit(train_datagen.flow(x_train, y_train_vec, batch_size=32),epochs=50)

Epoch 1/50
1563/1563 [==============================] - 44s 22ms/step - loss: 2.0989 - accuracy: 0.2051
Epoch 2/50
1563/1563 [==============================] - 34s 22ms/step - loss: 1.8620 - accuracy: 0.2808
Epoch 3/50
1563/1563 [==============================] - 34s 22ms/step - loss: 1.6748 - accuracy: 0.3703
Epoch 4/50
1563/1563 [==============================] - 34s 22ms/step - loss: 1.4820 - accuracy: 0.4599
Epoch 5/50
1563/1563 [==============================] - 34s 22ms/step - loss: 1.3467 - accuracy: 0.5204
Epoch 6/50
1563/1563 [==============================] - 34s 22ms/step - loss: 1.2284 - accuracy: 0.5679
Epoch 7/50
1563/1563 [==============================] - 34s 22ms/step - loss: 1.1347 - accuracy: 0.6067
Epoch 8/50
1563/1563 [==============================] - 34s 22ms/step - loss: 1.0604 - accuracy: 0.6328
Epoch 9/50
1563/1563 [==============================] - 34s 22ms/step - loss: 0.9998 - accuracy: 0.6559
Epoch 10/50
1563/1563 [==============================] - 34s 22m

In [ ]:
model_6.save('A2_VGG16_U34351756_Final1.keras')

#### ResNet (model 7) Re-training with same architecture (epochs = 25)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# entire training set
train_datagen.fit(x_train)

In [ ]:
from keras import optimizers

#learning_rate = 1e-4

learning_rate = 1e-3

model_7.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
history = model_7.fit(train_datagen.flow(x_train, y_train_vec, batch_size=32),epochs=25)

Epoch 1/25
1563/1563 [==============================] - 288s 166ms/step - loss: 1.9630 - accuracy: 0.2803
Epoch 2/25
1563/1563 [==============================] - 254s 163ms/step - loss: 1.7464 - accuracy: 0.3738
Epoch 3/25
1563/1563 [==============================] - 254s 163ms/step - loss: 1.5930 - accuracy: 0.4269
Epoch 4/25
1563/1563 [==============================] - 255s 163ms/step - loss: 1.4578 - accuracy: 0.4760
Epoch 5/25
1563/1563 [==============================] - 255s 163ms/step - loss: 1.3447 - accuracy: 0.5159
Epoch 6/25
1563/1563 [==============================] - 255s 163ms/step - loss: 1.2451 - accuracy: 0.5548
Epoch 7/25
1563/1563 [==============================] - 255s 163ms/step - loss: 1.1618 - accuracy: 0.5853
Epoch 8/25
1563/1563 [==============================] - 254s 163ms/step - loss: 1.1096 - accuracy: 0.6047
Epoch 9/25
1563/1563 [==============================] - 254s 163ms/step - loss: 1.0534 - accuracy: 0.6250
Epoch 10/25
1563/1563 [=======================

##### Model training is progressing very slowly - maybe a better learning rate would have improved the training process as mentioned here.
https://pyimagesearch.com/2019/09/23/keras-starting-stopping-and-resuming-training/



#### ResNet (model 7) Re-training with same architecture but a different learning rate (epochs = 50)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# entire training set
train_datagen.fit(x_train)

In [ ]:
from keras import optimizers

#learning_rate = 1e-4

learning_rate = 0.01

model_7.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
history = model_7.fit(train_datagen.flow(x_train, y_train_vec, batch_size=32),epochs=50)

Epoch 1/50
1563/1563 [==============================] - 284s 162ms/step - loss: 2.1477 - accuracy: 0.2294
Epoch 2/50
1563/1563 [==============================] - 254s 162ms/step - loss: 1.7922 - accuracy: 0.3270
Epoch 3/50
1563/1563 [==============================] - 254s 162ms/step - loss: 1.5967 - accuracy: 0.4145
Epoch 4/50
1563/1563 [==============================] - 254s 163ms/step - loss: 1.4653 - accuracy: 0.4666
Epoch 5/50
1563/1563 [==============================] - 254s 163ms/step - loss: 1.3762 - accuracy: 0.5005
Epoch 6/50
1563/1563 [==============================] - 254s 162ms/step - loss: 1.2989 - accuracy: 0.5305
Epoch 7/50
1563/1563 [==============================] - 254s 162ms/step - loss: 1.2330 - accuracy: 0.5593
Epoch 8/50
1563/1563 [==============================] - 254s 162ms/step - loss: 1.1743 - accuracy: 0.5798
Epoch 9/50
1563/1563 [==============================] - 254s 162ms/step - loss: 1.1249 - accuracy: 0.5982
Epoch 10/50
1563/1563 [=======================

##### __Lost connectivity__ :)
So I exhausted my compute units for the month for the A100 NVIDIA GPUs. In the last few epochs the accuracy improvement has slowed down to around 0.4% - which means that it will take a lot more iterations to converge to even 90%.
The Tesla T4 GPU takes 42 minutes to complete 1 epoch as opposed to just 4 minutes for 1 epoch on the A100s. So I'm stopping training for this model here.

#### VGG16 (model 6) Re-training with same architecture (epochs = 75)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# entire training set
train_datagen.fit(x_train)

In [ ]:
from keras import optimizers

#learning_rate = 1e-4

learning_rate = 1e-3

model_6.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
history = model_6.fit(train_datagen.flow(x_train, y_train_vec, batch_size=32),epochs=75)

Epoch 1/75
1563/1563 [==============================] - 43s 21ms/step - loss: 2.0789 - accuracy: 0.2104
Epoch 2/75
1563/1563 [==============================] - 33s 21ms/step - loss: 1.8831 - accuracy: 0.2821
Epoch 3/75
1563/1563 [==============================] - 34s 22ms/step - loss: 1.7189 - accuracy: 0.3486
Epoch 4/75
1563/1563 [==============================] - 34s 21ms/step - loss: 1.5728 - accuracy: 0.4098
Epoch 5/75
1563/1563 [==============================] - 33s 21ms/step - loss: 1.4231 - accuracy: 0.4890
Epoch 6/75
1563/1563 [==============================] - 34s 21ms/step - loss: 1.2810 - accuracy: 0.5483
Epoch 7/75
1563/1563 [==============================] - 34s 21ms/step - loss: 1.1837 - accuracy: 0.5897
Epoch 8/75
1563/1563 [==============================] - 33s 21ms/step - loss: 1.0935 - accuracy: 0.6218
Epoch 9/75
1563/1563 [==============================] - 34s 22ms/step - loss: 1.0323 - accuracy: 0.6474
Epoch 10/75
1563/1563 [==============================] - 34s 22m

In [ ]:
model_6.save('A2_VGG16_U34351756_Final2.keras')

#### VGG16 (model 6) Re-training with same architecture (epochs = 125)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# entire training set
train_datagen.fit(x_train)

In [ ]:
from keras import optimizers

#learning_rate = 1e-4

learning_rate = 1e-3

model_6.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
history = model_6.fit(train_datagen.flow(x_train, y_train_vec, batch_size=32),epochs=125)

Epoch 1/125
1563/1563 [==============================] - 67s 29ms/step - loss: 2.1100 - accuracy: 0.2010
Epoch 2/125
1563/1563 [==============================] - 46s 30ms/step - loss: 1.8460 - accuracy: 0.2922
Epoch 3/125
1563/1563 [==============================] - 46s 30ms/step - loss: 1.6733 - accuracy: 0.3682
Epoch 4/125
1563/1563 [==============================] - 46s 30ms/step - loss: 1.5501 - accuracy: 0.4283
Epoch 5/125
1563/1563 [==============================] - 46s 30ms/step - loss: 1.4135 - accuracy: 0.4968
Epoch 6/125
1563/1563 [==============================] - 47s 30ms/step - loss: 1.2956 - accuracy: 0.5483
Epoch 7/125
1563/1563 [==============================] - 46s 30ms/step - loss: 1.1927 - accuracy: 0.5915
Epoch 8/125
1563/1563 [==============================] - 46s 30ms/step - loss: 1.1072 - accuracy: 0.6182
Epoch 9/125
1563/1563 [==============================] - 46s 30ms/step - loss: 1.0312 - accuracy: 0.6459
Epoch 10/125
1563/1563 [==============================]

##### This model (125 epochs) does __not__ produce the highest accuracy with the test set (89%). The 75 epoch version does better (90%).

### Evaluate the model on the test set

#### VGG16 architecture - 50 epochs (model 6 performance on test set)
#### __Accuracy = 90.04%, Loss=0.317__

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_datagen.fit(x_test)

In [ ]:
loss_and_acc = model_6.evaluate(test_datagen.flow(x_test, y_test_vec))
print('loss = ' + str(loss_and_acc[0]))
print('accuracy = ' + str(loss_and_acc[1]))

313/313 [==============================] - 2s 4ms/step - loss: 0.3172 - accuracy: 0.9005
loss = 0.31722286343574524
accuracy = 0.9004999995231628
